Carry cost is [usually 15% to 30% of the inventory sale price](https://www.zoho.com/inventory/academy/inventory-management/what-is-carrying-cost.html#:~:text=Carrying%20costs%20are%20usually%2015,losing%20money%20over%20unsalable%20items). However, this is typically measured at a annual frequency, so the daily holding cost is more around 0.8% per day. For goods ordered in specifically for Black Friday, the holding cost is likely to be a bit higher due to the time-limited nature of the sales and discounts, so we will increase the daily holding cost to about 2%.
Shortage cost is close to the margin of the good, ie. a lost sale. [50-70% are considered good margins](https://www.unleashedsoftware.com/inventory-accounting-guide/what-is-a-good-profit-margin/).

# We have four scenarios:
- High truck cost, weekly scheduling
- High truck cost, daily scheduling (with a 50% daily premium paid for the luxury)
- Low truck cost, weekly scheduling
- Low truck cost, daily scheduling (with a 50% daily premium paid for the luxury)

Furthermore, we will assume that due to driver constraints, the company has access to only 10 drivers on any day, and hence can hire at most 10 trucks.

Our three items are based on the following high-demand Black Friday items:
- A [new phone launch](https://www.jbhifi.com.au/products/samsung-galaxy-s25-ultra-512gb-titanium-black)
    - Retails for 2200.
    - Which makes the shortage cost roughly 1100.
    - Daily holding cost of 2% is 44.
    - Has a [physical size](https://www.samsung.com/au/smartphones/galaxy-s25-ultra/specs/#:~:text=Physical%20specification,Gear%20Support) of 162.8mm x 77.6mm x 8.2mm, so the box is going to be around 220mm x 130mm x 50mm, meaning this item is around 0.00143 m3, so we could fit around 700 phones per cubic metre.
    - The demand for this item will be relatively high, but due to its high price we will give it a muted demand profile (Good 3).
- A [heavily discounted pair of production recording headphones](https://www.jbhifi.com.au/products/audio-technica-ath-r50x-professional-open-back-headphone)
    - Retails for 400.
    - Which makes the shortage cost about 200.
    - And the daily holding cost 8.
    - Has a [physical size](https://www.amazon.com/Audio-Technica-Professional-Open-Back-Reference-Headphones/dp/B0DTTWF1Z9?th=1) of 100mm x 170mm x 200mm, box size roughly 150mm x 220 mm x 250 mm, which is 0.00825m3, so we could fit about 120 headphones per cubic metre.
    - The demand for this item will be medium, the price point isn't too high and we can assume a large discount is being applied for Black Friday, but it is definitely a more niche product. We will give it the spiky profile (Good 2).
- A [big discount on Samsung's previous model Galaxy Buds](https://www.jbhifi.com.au/products/samsung-galaxy-buds-fe-white)
    - Retails for 200.
    - This would make the shortage cost around 100, but let's say there are significant discounts on this product for Black Friday as its being replaced by the Galaxy Buds 2 in a few months, so the shortage comes down to 50.
    - The daily holding cost would be 4, however because of the heavy discounts applied and upcoming obsolescence, it is unlikely unsold goods will sell. So we will increase the holding cost for this item to 10 to reflect the risk of overstocking.
    - The [box size](https://www.amazon.com.au/Samsung-Galaxy-Buds-FE-SM-R400NZWAEUA/dp/B0CPFH7FBM) is tiny at 97 mm x 85 mm x 42 mm, which is 0.00034629 m3, or about 3K items per cubic metre.


Weekly truck rental data from [here](https://www.budgettrucks.com.au/en/offers/vic/weekly-rates).

1100 for 7 day truck rental with 50m3 space sourced from [here](https://www.hertztrucks.com.au/bookings/choose-vehicle).
- Reasonable to say that daily truck rental cost would be between 100 and 200 depending on demand.
- We are going to assume that not all the capacity of the truck is actually available for just transporting our three items, that would be a ridiculous scenario.
- For instance, we can fit 700 phones into one cubic metre, so the full truck could transport 35K phones.
- Let's make an assumption that these three goods are among 300 items being transported for Black Friday deals, and these items are representative both in price and size. Then we can assume that the average space within each truck used for these items would be 1% of the total capacity, or half a cubic metre.
- Thus we could fit 350 phones per truck, 60 headphones and 1.5K earphones. If we set the truck capacity as the LCM of these values, then we can set the size of each item as:
    - LCM: 10,500
        - 350 phones per truck = size of 30
        - 60 headphones per truck = size of 175
        - 1500 earphones per truck = size of 7

Thus, we end with the following specifications:

Good 1
    - Name: Earphones
    - Size: 7
    - Shortage cost: 50
    - Holding cost: 10

Good 2
    - Name: Headphones
    - Size: 175
    - Shortage cost: 200
    - Holding cost: 8

Good 3
    - Name: Phone
    - Size: 30
    - Shortage cost: 1100
    - Holding cost: 44

Truck
    - Capacity: 10,500
    - Maximum daily trucks: 10
    - Daily cost (low): 100
    - Daily cost (high): 200
    - Premium for daily hire (instead of weekly): 50%

In [1]:
from typing import Literal, Any, Optional
from dataclasses import dataclass
import pandas as pd
import numpy as np
import gurobipy as gp
import json

# dataclass to store our full set of parameters
@dataclass
class ScenarioParams:
    # Provided by scale of problem in `data/scales` folder
    num_warehouses: int
    num_goods: int
    num_retailers: int

    # Provided by scenario in `data/scenarios` folder
    num_periods: int
    truck_capacity: int
    truck_daily_rental_cost: int
    truck_rental_period: Literal["daily", "weekly"]

    def __post_init__(self):
        # Derived quantities and synthetic data
        self.good_sizes: np.array = np.random.randint(1, 20, self.num_goods)
        self.holding_costs: np.array = np.random.randint(20, 400, self.num_goods)
        self.shortage_costs = self.holding_costs * np.random.uniform(1, 1.2, size=self.num_goods)
        self.transportation_costs = np.round(np.random.uniform(1, 4, (self.num_warehouses, self.num_retailers, self.num_goods)), 1)
        self.retailer_good_demand = np.round(np.random.randint(40, 100, (self.num_retailers, self.num_goods, self.num_periods)), 1)

        # Black Friday simulation: apply scaling factor of [1.0, 0.7, 0.3, 0.1, 4.0, 2.0, 1.0] to demand profile
        for i in range(self.num_retailers):
            for j in range(self.num_goods):
                self.retailer_good_demand[i][j] = np.multiply(self.retailer_good_demand[i][j], [1.0, 0.7, 0.4, 0.2, 3.0, 1.5, 1.0])

        self.max_trucks: int = int(1.2 * self.good_sizes @ np.sum(self.retailer_good_demand, axis=(0, 2)) / self.num_periods)

In [2]:
def solve_model(
    scenario_parameters: ScenarioParams,
    opt_param_dict: Optional[dict[str, Any]] = None,
    branch: bool = False
) -> dict:
    m = gp.Model()
    #m.setParam('OutputFlag', 0)

    if opt_param_dict is not None:
        for key, value in opt_param_dict.items():
            m.setParam(key, value)

    max_trucks = scenario_parameters.max_trucks
    truck_capacity = scenario_parameters.truck_capacity
    truck_daily_rental_cost = scenario_parameters.truck_daily_rental_cost
    truck_rental_period = scenario_parameters.truck_rental_period
    good_sizes = scenario_parameters.good_sizes
    holding_costs = scenario_parameters.holding_costs
    shortage_costs = scenario_parameters.shortage_costs
    transportation_costs = scenario_parameters.transportation_costs
    retailer_good_demand = scenario_parameters.retailer_good_demand

    num_periods = scenario_parameters.num_periods
    num_warehouses = scenario_parameters.num_warehouses
    num_goods = scenario_parameters.num_goods
    num_retailers = scenario_parameters.num_retailers

    warehouses = range(scenario_parameters.num_warehouses)
    goods = range(scenario_parameters.num_goods)
    retailers = range(scenario_parameters.num_retailers)
    periods = range(scenario_parameters.num_periods)

    if truck_rental_period == "weekly":
        num_trucks = m.addVar(lb=0, ub=max_trucks, vtype='I', name="NumTrucks")
        if branch:
            num_trucks.BranchPriority = 100
    elif truck_rental_period == "daily":
        num_trucks = m.addMVar(num_periods, lb=0, ub=max_trucks, vtype='I', name="NumTrucks")
        if branch:
            num_trucks.BranchPriority = 100
    else:
        raise NotImplementedError("Only supports `weekly` or `daily` truck rental.")

    truck_allocation = m.addMVar((num_warehouses, num_periods), lb=0, ub=1000, vtype='I', name="TruckAllocation")
    transport = m.addMVar((num_warehouses, num_retailers, num_goods, num_periods), lb=0, ub=10000, vtype='I', name="TransportedGoods")
    carried_retailer_stock = m.addMVar((num_retailers, num_goods, num_periods+1), lb=0, ub=10000, vtype='I', name="RetailerHeldStock")
    short_retailer_stock = m.addMVar((num_retailers, num_goods, num_periods), lb=0, ub=10000, vtype='I', name="RetailerShortStock")

    if truck_rental_period == "weekly":
        m.setObjective(
            gp.quicksum(transport[w, r, g, p] * transportation_costs[w, r, g] for w in warehouses for r in retailers for g in goods for p in periods) +
            gp.quicksum(short_retailer_stock[r, g, p] * shortage_costs[g] for g in goods for r in retailers for p in periods) +
            gp.quicksum(carried_retailer_stock[r, g, p] * holding_costs[g] for g in goods for r in retailers for p in range(num_periods+1)) +
            num_trucks * truck_daily_rental_cost * num_periods,
            gp.GRB.MINIMIZE
        )

        # total number of trucks allocated across all warehouses for each period must equal number of trucks hired
        m.addConstrs((gp.quicksum(truck_allocation[w, p] for w in warehouses) == num_trucks for p in periods), name="TruckAllocation")

    elif truck_rental_period == "daily":
        m.setObjective(
            gp.quicksum(transport[w, r, g, p] * transportation_costs[w, r, g] for w in warehouses for r in retailers for g in goods for p in periods) +
            gp.quicksum(short_retailer_stock[r, g, p] * shortage_costs[g] for g in goods for r in retailers for p in periods) +
            gp.quicksum(carried_retailer_stock[r, g, p] * holding_costs[g] for g in goods for r in retailers for p in range(num_periods+1)) +
            gp.quicksum(num_trucks[p] for p in periods) * truck_daily_rental_cost,
            gp.GRB.MINIMIZE
        )
        # total number of trucks allocated across all warehouses for each period must equal number of trucks hired
        m.addConstrs((gp.quicksum(truck_allocation[w, p] for w in warehouses) == num_trucks[p] for p in periods), name="TruckAllocation")
    else:
        raise NotImplementedError("Only supports `weekly` or `daily` truck rental.")


    # total goods sent from a warehouse in each period cannot exceed the capacity of its assigned trucks
    m.addConstrs((gp.quicksum(gp.quicksum(transport[w, r, g, p] for r in retailers) * good_sizes[g] for g in goods) <= truck_allocation[w, p] * truck_capacity for w in warehouses for p in periods), name="TruckCapacities")

    # first period carried retailer stock equal to 0
    m.addConstrs((carried_retailer_stock[r, g, 0] == 0 for r in retailers for g in goods), name="InitialRetailerStock")

    # amount of retailer goods carried each period equal to the excess demand in each period
    m.addConstrs(
        (
            carried_retailer_stock[r, g, p + 1] == gp.quicksum(transport[w, r, g, p] for w in warehouses) - retailer_good_demand[r, g, p] + carried_retailer_stock[r, g, p] + short_retailer_stock[r, g, p] for r in retailers for g in goods for p in periods
        ), name="RetailerCarryAmounts"
    )

    # amount of retailer goods short of demand is equal to the unmet demand in each period
    m.addConstrs(
        (
            short_retailer_stock[r, g, p] >= retailer_good_demand[r, g, p] - carried_retailer_stock[r, g, p] - gp.quicksum(transport[w, r, g, p] for w in warehouses) for r in retailers for g in goods for p in periods
        ), name="RetailerShortAmounts"
    )

    m.optimize()

    retail_stock_output = get_retail_stock_traces(
        scenario_parameters=scenario_parameters,
        carried_retailer_stock=carried_retailer_stock,
        transport=transport,
        retailer_good_demand=retailer_good_demand,
        short_retailer_stock=short_retailer_stock,
    )
    truck_utilisation = get_truck_utilisation(
        scenario_parameters=scenario_parameters,
        truck_allocation=truck_allocation,
        transport=transport
    )
    cost_attribution = get_cost_attribution(
        scenario_parameters=scenario_parameters,
        num_trucks=num_trucks,
        short_retailer_stock=short_retailer_stock,
        carried_retailer_stock=carried_retailer_stock,
        transport=transport,
        m=m,
    )

    return {
        'optimised_model': m,
        'retail_stock_output': retail_stock_output,
        'truck_utilisation': truck_utilisation,
        'cost_attribution': cost_attribution
    }

In [3]:
def get_retail_stock_traces(
    scenario_parameters: ScenarioParams,
    carried_retailer_stock,
    transport,
    retailer_good_demand,
    short_retailer_stock,
) -> list:
    output = []
    goods = range(scenario_parameters.num_goods)
    retailers = range(scenario_parameters.num_retailers)
    warehouses = range(scenario_parameters.num_warehouses)
    periods = range(scenario_parameters.num_periods)

    for retailer in retailers:
        for good in goods:
            for period in periods:
                output.append(
                    {
                        "Retailer": f"Retail{retailer}",
                        "Good": f"Good{good}",
                        "Period": period,
                        "InitialStock": carried_retailer_stock[retailer][good][period].x.item() + 0.0,
                        "DailySupply": sum(transport[w, retailer, good, period].x + 0.0 for w in warehouses),
                        "Demand": retailer_good_demand[retailer][good][period],
                        "ShortStock": short_retailer_stock[retailer][good][period].x.item() + 0.0,
                    }
                )

    return output

In [4]:
def get_truck_utilisation(
    scenario_parameters: ScenarioParams,
    truck_allocation,
    transport,
) -> list:
    output = []

    warehouses = range(scenario_parameters.num_warehouses)
    periods = range(scenario_parameters.num_periods)
    retailers = range(scenario_parameters.num_retailers)
    goods = range(scenario_parameters.num_goods)

    for warehouse in warehouses:
        for period in periods:
            output.append(
                {
                    'Warehouse': f"Warehouse{warehouse}",
                    'Period': period,
                    'AssignedTrucks': truck_allocation[warehouse, period].x + 0.0,
                    'TotalCapacity': (truck_allocation[warehouse, period].x + 0.0) * scenario_parameters.truck_capacity,
                    'TotalSupplied': sum(transport[warehouse, r, g, period].x * scenario_parameters.good_sizes[g] + 0.0 for r in retailers for g in goods)
                }
            )

    return output

In [5]:
def get_cost_attribution(
    scenario_parameters: ScenarioParams,
    num_trucks,
    short_retailer_stock,
    carried_retailer_stock,
    transport,
    m,
) -> dict:
    warehouses = range(scenario_parameters.num_warehouses)
    retailers = range(scenario_parameters.num_retailers)
    goods = range(scenario_parameters.num_goods)
    periods = range(scenario_parameters.num_periods)

    if scenario_parameters.truck_rental_period == "weekly":
        output = {
            'NumTrucks': num_trucks.x,
            'TruckHire': num_trucks.x * scenario_parameters.truck_daily_rental_cost * scenario_parameters.num_periods,
            'TransportCosts': sum(transport[w, r, g, p].x * scenario_parameters.transportation_costs[w, r, g] for w in warehouses for r in retailers for g in goods for p in periods),
            'ShortageCosts': sum(short_retailer_stock[r, g, p].x * scenario_parameters.shortage_costs[g] for g in goods for r in retailers for p in periods),
            'HoldingCosts': sum(carried_retailer_stock[r, g, p].x * scenario_parameters.holding_costs[g] for g in goods for r in retailers for p in range(scenario_parameters.num_periods + 1)),
            'TotalCost': m.objVal
        }
    elif scenario_parameters.truck_rental_period == "daily":
        output = {
            'NumTrucks': list(num_trucks.x + 0.0),
            'TruckHireCosts': sum(num_trucks.x) * scenario_parameters.truck_daily_rental_cost,
            'TransportCosts': sum(transport[w, r, g, p].x * scenario_parameters.transportation_costs[w, r, g] + 0.0 for w in warehouses for r in retailers for g in goods for p in periods),
            'ShortageCosts': sum(short_retailer_stock[r, g, p].x * scenario_parameters.shortage_costs[g] for g in goods for r in retailers for p in periods),
            'HoldingCosts': sum(carried_retailer_stock[r, g, p].x * scenario_parameters.holding_costs[g] for g in goods for r in retailers for p in range(scenario_parameters.num_periods + 1)),
            'TotalCost': m.objVal
        }
    else:
        output = {}

    return output

In [ ]:
stats = []

# my computer can't solve anything bigger than "small" within 30 minutes...
for scale in ["very_small", "small"]:#, "medium", "large"]:
    for scenario in ["daily_trucks_high_cost", "daily_trucks_low_cost", "weekly_trucks_high_cost", "weekly_trucks_low_cost"]:
        with open(f"../data/scenarios/{scenario}.json", 'r') as scenario_file:
            scenario_input = json.load(scenario_file)

        with open(f"../data/scales/{scale}.json", "r") as scale_file:
            scale_input = json.load(scale_file)

        scenario_params = ScenarioParams(**{**scenario_input, **scale_input})

        solution_dict = solve_model(
            scenario_parameters=scenario_params,
            opt_param_dict=None,
            branch=False
        )

        stats.append(
            {
                'scenario': scenario,
                'scale': scale,
                'retail_stock_output': solution_dict['retail_stock_output'],
                'truck_utilisation': solution_dict['truck_utilisation'],
                'cost_attribution': solution_dict['cost_attribution']
            }
        )

        print(f"{scenario} | {scale} | completed")

Set parameter Username
Set parameter LicenseID to value 2636594
Academic license - for non-commercial use only - expires 2026-03-14
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 9 7950X 16-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 267 rows, 792 columns and 2192 nonzeros
Model fingerprint: 0x54aee992
Variable types: 0 continuous, 792 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 3e+02]
  Bounds range     [1e+03, 1e+04]
  RHS range        [8e+00, 3e+02]
Found heuristic solution: objective 320730.12336
Presolve removed 127 rows and 22 columns
Presolve time: 0.00s
Presolved: 140 rows, 770 columns, 1385 nonzeros
Variable types: 0 continuous, 770 integer (0 binary)

Root relaxation: objective 1.428543e+04, 158 iterations, 0.00 seconds (0.00 work units)

    Nodes  

In [ ]:
def get_retail_stock_df(
    retail_stock: list
) -> pd.DataFrame:

    return pd.DataFrame.from_records(retail_stock)

In [ ]:
def get_truck_utilisation_df(
    truck_utilisation: list
) -> pd.DataFrame:

    truck_utilisation_df = pd.DataFrame.from_records(truck_utilisation)

    truck_utilisation_df['Utilisation'] = (truck_utilisation_df['TotalSupplied'] / truck_utilisation_df['TotalCapacity']).fillna(1.0).apply(lambda x: "{:.2f}".format(x))

    return truck_utilisation_df

In [ ]:
def get_cost_attribution_df(
    cost_attribution: dict
) -> pd.DataFrame:
    return pd.DataFrame(cost_attribution, index=[0])

In [ ]:
# example output
get_retail_stock_df(stats[0]['retail_stock_output'])

In [ ]:
get_truck_utilisation_df(stats[0]['truck_utilisation'])

In [ ]:
get_cost_attribution_df(stats[0]['cost_attribution'])